# TFM Suministro Global - CREAR RELACIONES: 
##           relación:  Grupos Aplicación/usos --[ INCLUIDO_EN ] -->  Sectores NACE2
##           relación:  Materiales y Grupos  --[ USADO_EN] --> Aplicación/usos  

# _Neo4j - py2neo_


## Partimos de la existencia, en paralelo a esta ejecución, de una instancia levantada y accesible de Neo4j  
### La primera vez que se quiera usar _py2neo_ erá  necesario asegurar su instalación:  _!pip install py2neo_

In [1]:
##!pip install py2neo 

In [1]:
#importar la librería para poder hacer uso de sus funcionalidades y objetos
from py2neo import *

In [2]:
#acceder a la instancia levantada de Neo4j - en nuestro caso instancia Docker - asignamos credenciales de autentificación -
graph_test01 = Graph(auth=('neo4j', 's3cr3t'))

In [3]:
#que es el objeto - graph_test01
graph_test01

Graph('bolt://localhost:7687')

In [4]:
#que es el objeto - tx
tx = graph_test01.begin()

In [5]:
tx

In [7]:
#query de conexión de prueba sin errores
# verificar existencia de NODOS de METERIALES Brutos y Refinados
#--- cuenta los nodos con diferentes tipos de etiqueta y los une en una única salida, 1 fila por cada tipo (name, cnt)
queryPRB= "MATCH (n)                                    \
RETURN labels (n)[0] as name, count (n) as Ntotal       \
UNION MATCH (n)                                         \
WITH labels (n)[1] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[2] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[3] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
"
graph_test01.run(queryPRB).to_data_frame()

,name,Ntotal
0,MaterialB,78
1,MaterialREF,78
2,Pais,91
3,GrupoAplicacion,88
4,SectorAplicacion,25


## CARGAR DATOS

In [9]:
import numpy as np    #datos y operativa álgegra y matrices de n dimensiones (arrays)
import pandas as pd   #carga dataset y manipulación de datos  y dataframes

In [10]:
#DATOS: Fichero completo de MATERIAL -> APLICACIONES/USOS -> SECTOR NACE2 APLICACIÓN, dentro del análisis 
# de suministro crítico GLOBAL actual  - Atendiendo al alcance del estudio - EU CRM 2020
#FICHERO: GLAplic_Cargar.csv
#CONTENIDO: Cada Material, su uso y cuota (share) de participación en Grupo de Aplicación/uso, y sector Nace2 vinculado
#           a cada  aplicación/uso   
#USO: Crear las RELACIONES entre "GrupoAplicacion" --[INCLUIDO_EN]--> "SectorAplicacion" y
#     ** Propiedades de [INCLUIDO_EN]: 
#           SHR: proporción decimal de2 dígitos (entre 0 y 1)
#           VAM (importe proporcional de valor añadido - calculado-): SHR por total de Valor Añadido del sector vinculado)   
#     Crear las RELACIONES entre "MaterialREF" --[USADO_EN]--> "GrupoAplicacion" 
#     ** esta relación se crea inicialmente sin propiedades
#
rutaFichero =  "https://raw.githubusercontent.com/JuanKFD/UNIR_TFM/main/GLAplic_Cargar.csv" 
dfRelAplics = pd.read_csv(rutaFichero, sep=';')


In [11]:
dfRelAplics.head()

,MaterialApl,ApplicationORG,ShareSectorApl,SectorAplMatORG,ValorAnadidoSector,SectorAplicacion,GrupoAplicacion
0,Aluminium,Construction,"0,23",C25 - Manufacture of fabricated metal products...,"148,35",C25-FabrMetalProds-NoMachin-Eq,Building & roads
1,Aluminium,Mobility (Transport and Automotive),"0,21","C29 - Manufacture of motor vehicles, trailers ...","160,60","C29-MotorVehicle,Trailer&SemiT",Transport (automotive)
2,Aluminium,Mobility (Transport and Automotive),"0,21",C30 - Manufacture of other transport equipment,"44,30",C30-other transport equipment,Transport (automotive)
3,Aluminium,Packaging,"0,17",C25 - Manufacture of fabricated metal products...,"148,35",C25-FabrMetalProds-NoMachin-Eq,Packaging
4,Aluminium,High Tech Engineering,"0,12",C28 - Manufacture of machinery and equipment n...,"182,59",C28-Machinery &Equipment n.e.c,High Tech Engineering


In [12]:
dfRelAplics.tail()

,MaterialApl,ApplicationORG,ShareSectorApl,SectorAplMatORG,ValorAnadidoSector,SectorAplicacion,GrupoAplicacion
437,Zirconium,Foundry,"0,15",C24 - Manufacture of basic metals,"55,43",C24-basic metals,Foundry
438,Zirconium,Chemicals,"0,12",C20 - Manufacture of chemicals and chemical pr...,"105,51",C20-Chemicals & Chemc.Prods.,Chemicals
439,Zirconium,Others,"0,10","C26 - Manufacture of computer, electronic and ...","65,70","C26-Computer,Electron.Optic.Ps",Chemicals
440,Zirconium,Pigments,"0,03",C20 - Manufacture of chemicals and chemical pr...,"105,51",C20-Chemicals & Chemc.Prods.,"Chemicals (Paint, Adhesives)"
441,Zirconium,"Superalloys, Nuclear","0,02",C24 - Manufacture of basic metals,"55,43",C24-basic metals,Energy (Nuclear)


In [13]:
dfRelAplics.shape

(442, 7)

In [14]:
dfRelAplics.columns

Index(['MaterialApl', 'ApplicationORG', 'ShareSectorApl', 'SectorAplMatORG',
       'ValorAnadidoSector', 'SectorAplicacion', 'GrupoAplicacion'],
      dtype='object')

### Cargados en memoria las _442 líneas de relación_  entre _material-uso-sector_, para **datos de la cadena de suministro global**

# PREVISUALIZACION DE DATOS - USOS

### Con el fin de procesar de manera más parametrizada y como presentación de otra manera de tratarlo, creo bucle en base a los propios datos de las columnas

In [15]:
strSeparator = "|"
for i in range(0, dfRelAplics.shape[0]):
    strVar = ""
    for j in range(0, dfRelAplics.shape[1]):
        strVar += dfRelAplics[dfRelAplics.columns[j]][i]
        strVar += strSeparator
    print(strVar) 
# se visualiza con el separador específico identificado en la variable  - strSeparator

Aluminium|Construction|0,23|C25 - Manufacture of fabricated metal products, except machinery and equipment|148,35|C25-FabrMetalProds-NoMachin-Eq|Building & roads|
Aluminium|Mobility (Transport and Automotive)|0,21|C29 - Manufacture of motor vehicles, trailers and semi-trailers|160,60|C29-MotorVehicle,Trailer&SemiT|Transport (automotive)|
Aluminium|Mobility (Transport and Automotive)|0,21|C30 - Manufacture of other transport equipment|44,30|C30-other transport equipment |Transport (automotive)|
Aluminium|Packaging|0,17|C25 - Manufacture of fabricated metal products, except machinery and equipment|148,35|C25-FabrMetalProds-NoMachin-Eq|Packaging|
Aluminium|High Tech Engineering|0,12|C28 - Manufacture of machinery and equipment n.e.c.|182,59|C28-Machinery &Equipment n.e.c|High Tech Engineering|
Aluminium|Consumer Durables|0,06|C25 - Manufacture of fabricated metal products, except machinery and equipment|148,35|C25-FabrMetalProds-NoMachin-Eq|Consumer Durables|
Antimony|Flame retardants|0,4

# CREAR RELACION ENTRE Grupo Aplicación y Sector Aplicación
### RELACIÓN: _GrupoAplicacion --[INCLUIDO_EN]--> "SectorAplicacion_
**Observaciones:**   
(1) Nodos previamente creados   
(2) Relación creada sin propiedades

### Consulta patrón _CQL-Neo4j_ para crear la relación buscada  
**MATCH**   
    (a:**GrupoAplicacion**),  
    (b:**SectorAplicacion**)  

**WHERE** a.name = _"grupoxxx"_ **AND** b.name = _"sectorxxx"_  
**CREATE** (a)**-[r:INCLUIDO_EN]->**(b)  
**RETURN** type(r)  

In [16]:
nFilas = dfRelAplics.shape[0]
nFilas 

442

In [20]:
#  CrearRELACION INCLUIDO_EN -  Una fila por cada relación, contiene el GrupoAplicacion y el SectorAplicacion 
# (previamente creados ambos nodos) 
#  Nodos: GrupoAplicacion -- para usos asignables a cada material refinado-procesado obtenido
## PREVISAULIZAR QUERYs antes de la eje# cución
TipoNodoA = "(a:GrupoAplicacion)"
TipoNodoB = "(b:SectorAplicacion)"
TipoRelacion = " :INCLUIDO_EN" 
inicio = 0
nFilas = 2
for i in range(inicio, nFilas):
    strQry = "MATCH "                                        \
            + TipoNodoA + ", " + TipoNodoB                   \
            + " WHERE a.name = '"                             \
            +  dfRelAplics.GrupoAplicacion[i]                \
            + "' AND b.name = '"                             \
            +  dfRelAplics.SectorAplicacion[i]               \
            + "'"                                           \
            + " CREATE (a)-[r" + TipoRelacion + "]->(b)"      \
            + " RETURN type(r)"                                 
    print("QRY [" + str(i) + "]> "+ strQry)
# querys  visualizadas

QRY [0]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Building & roads' AND b.name = 'C25-FabrMetalProds-NoMachin-Eq' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [1]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Transport (automotive)' AND b.name = 'C29-MotorVehicle,Trailer&SemiT' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)


In [21]:
#comprobar que sigue abierta la conexión para enviar las queries a la BD 
graph_test01

Graph('bolt://localhost:7687')

In [19]:
# Crear la RELACIÓN en la BD , una por CADA FILA 
#relación INCLUIDO_EN
## query para crea la relación con sus propiedades (en este caso no aplica)
inicio = 0
nFilas = 2

for i in range(inicio, nFilas):
    strQry = "MATCH "                                        \
            + TipoNodoA + ", " + TipoNodoB                   \
            + " WHERE a.name = '"                             \
            +  dfRelAplics.GrupoAplicacion[i]                \
            + "' AND b.name = '"                             \
            +  dfRelAplics.SectorAplicacion[i]               \
            + "'"                                           \
            + " CREATE (a)-[r" + TipoRelacion + "]->(b)"      \
            + " RETURN type(r)"                                 
#
# ejecutar Query en BD abierta - Neo4j
#
    graph_test01.run(strQry)
#Visualizar Query ejecutada 
    print("EJECUTADA QRY [" + str(i) + "]> "+ strQry)
# nodos creados

EJECUTADA QRY [0]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Building & roads' AND b.name = 'C25-FabrMetalProds-NoMachin-Eq' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [1]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Transport (automotive)' AND b.name = 'C29-MotorVehicle,Trailer&SemiT' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)


In [24]:
dfGraphMat= graph_test01.run("MATCH p=()-[r:INCLUIDO_EN]->() RETURN p LIMIT 25").data()

In [25]:
dfGraphMat


[{'p': Path(Node('GrupoAplicacion', name='Transport (automotive)'), INCLUIDO_EN(Node('GrupoAplicacion', name='Transport (automotive)'), Node('SectorAplicacion', ValorAnadidoSector=160.6, name='C29-MotorVehicle,Trailer&SemiT')))},
 {'p': Path(Node('GrupoAplicacion', name='Building & roads'), INCLUIDO_EN(Node('GrupoAplicacion', name='Building & roads'), Node('SectorAplicacion', ValorAnadidoSector=148.35, name='C25-FabrMetalProds-NoMachin-Eq')))}]

### Creamos el resto de las relaciones  
una vez que se verifica la creación de las 2 primeras filas

In [26]:
nFilas

2

In [27]:
# Crear la RELACIÓN en la BD , una por CADA FILA (****restantes ***) 
#relación INCLUIDO_EN
## query para crea la relación con sus propiedades (en este caso no aplica)
inicio = nFilas
nFilas = dfRelAplics.shape[0]
for i in range(2, nFilas):
    strQry = "MATCH "                                        \
            + TipoNodoA + ", " + TipoNodoB                   \
            + " WHERE a.name = '"                             \
            +  dfRelAplics.GrupoAplicacion[i]                \
            + "' AND b.name = '"                             \
            +  dfRelAplics.SectorAplicacion[i]               \
            + "'"                                           \
            + " CREATE (a)-[r" + TipoRelacion + "]->(b)"      \
            + " RETURN type(r)"                                 
    print("QRY [" + str(i) + "]> "+ strQry)
#
# ejecutar Query en BD abierta - Neo4j
#
    graph_test01.run(strQry)
#Visualizar Query ejecutada 
    print("EJECUTADA QRY [" + str(i) + "]> "+ strQry)
# nodos creados

QRY [2]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Transport (automotive)' AND b.name = 'C30-other transport equipment ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [2]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Transport (automotive)' AND b.name = 'C30-other transport equipment ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [3]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Packaging' AND b.name = 'C25-FabrMetalProds-NoMachin-Eq' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [3]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Packaging' AND b.name = 'C25-FabrMetalProds-NoMachin-Eq' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [4]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'High Tech Engineering' AND b.name = 'C28-Machinery &Equipment n.e.c' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [4]> MATCH (a:GrupoAplicacion), (b:Sec

QRY [27]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Foods' AND b.name = 'C11-beverages ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [27]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Foods' AND b.name = 'C11-beverages ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [28]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Metals-Basics' AND b.name = 'C24-basic metals ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [28]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Metals-Basics' AND b.name = 'C24-basic metals ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [29]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Chemicals' AND b.name = 'C20-Chemicals & Chemc.Prods.' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [29]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Chemicals' AND b.name = 'C20-Chemicals & Chemc.Prods.' CREATE

EJECUTADA QRY [56]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Catalysts' AND b.name = 'C20-Chemicals & Chemc.Prods.' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [57]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Ceramics & Frits' AND b.name = 'C23-Oth.NoMetal.mineral Prods.' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [57]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Ceramics & Frits' AND b.name = 'C23-Oth.NoMetal.mineral Prods.' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [58]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Electrical and Electronics' AND b.name = 'C26-Computer,Electron.Optic.Ps' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [58]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Electrical and Electronics' AND b.name = 'C26-Computer,Electron.Optic.Ps' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [59]> MATCH (a:GrupoAplic

EJECUTADA QRY [89]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Industry' AND b.name = 'C28-Machinery &Equipment n.e.c' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [90]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Foods' AND b.name = 'C11-beverages ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [90]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Foods' AND b.name = 'C11-beverages ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [91]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Mining-Extraction' AND b.name = 'C23-Oth.NoMetal.mineral Prods.' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [91]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Mining-Extraction' AND b.name = 'C23-Oth.NoMetal.mineral Prods.' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [92]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Activated raw granules' AN

EJECUTADA QRY [129]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Building & roads' AND b.name = 'C23-Oth.NoMetal.mineral Prods.' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [130]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Building & roads' AND b.name = 'C23-Oth.NoMetal.mineral Prods.' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [130]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Building & roads' AND b.name = 'C23-Oth.NoMetal.mineral Prods.' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [131]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Building & roads' AND b.name = 'C23-Oth.NoMetal.mineral Prods.' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [131]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Building & roads' AND b.name = 'C23-Oth.NoMetal.mineral Prods.' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [132]> MATCH (a:GrupoAplicacion

...

EJECUTADA QRY [278]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Special Steel' AND b.name = 'C30-other transport equipment ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [279]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Catalysts' AND b.name = 'C29-MotorVehicle,Trailer&SemiT' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [279]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Catalysts' AND b.name = 'C29-MotorVehicle,Trailer&SemiT' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [280]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Electrical and Electronics' AND b.name = 'C26-Computer,Electron.Optic.Ps' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [280]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Electrical and Electronics' AND b.name = 'C26-Computer,Electron.Optic.Ps' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [281]> MATCH (a:GrupoAplicac

EJECUTADA QRY [320]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Glass' AND b.name = 'C23-Oth.NoMetal.mineral Prods.' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [321]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Chemicals' AND b.name = 'C20-Chemicals & Chemc.Prods.' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [321]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Chemicals' AND b.name = 'C20-Chemicals & Chemc.Prods.' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [322]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Others' AND b.name = 'Z99-Others**' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [322]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Others' AND b.name = 'Z99-Others**' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [323]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Electrical and Electronics' AND b.name = 'C26

EJECUTADA QRY [344]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Chemicals' AND b.name = 'C23-Oth.NoMetal.mineral Prods.' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [345]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Foundry' AND b.name = 'C24-basic metals ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [345]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Foundry' AND b.name = 'C24-basic metals ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [346]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Filler, extender, sealant' AND b.name = 'C22-rubber & plastic prods. ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [346]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Filler, extender, sealant' AND b.name = 'C22-rubber & plastic prods. ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [347]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name 

EJECUTADA QRY [372]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Chemicals (Paper)' AND b.name = 'C17-paper & paper prods. ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [373]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Chemicals (Plastics)' AND b.name = 'C22-rubber & plastic prods. ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [373]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Chemicals (Plastics)' AND b.name = 'C22-rubber & plastic prods. ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [374]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Chemicals (Paper)' AND b.name = 'C17-paper & paper prods. ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [374]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Chemicals (Paper)' AND b.name = 'C17-paper & paper prods. ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [375]> MATCH (a:GrupoAplicacion), (b:Se

QRY [408]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Alloys' AND b.name = 'C24-basic metals ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [408]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Alloys' AND b.name = 'C24-basic metals ' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [409]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Pharmacy' AND b.name = 'C21-BasicPharmaceutic.&PhPreps' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [409]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Pharmacy' AND b.name = 'C21-BasicPharmaceutic.&PhPreps' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
QRY [410]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Industry' AND b.name = 'C28-Machinery &Equipment n.e.c' CREATE (a)-[r :INCLUIDO_EN]->(b) RETURN type(r)
EJECUTADA QRY [410]> MATCH (a:GrupoAplicacion), (b:SectorAplicacion) WHERE a.name = 'Industry' AND b.name = 'C28-Ma

In [28]:
#validamos que todos los nodos se han creado (hasta este paso)
queryPRB= "MATCH (n)                                    \
RETURN labels (n)[0] as name, count (n) as Ntotal       \
UNION MATCH (n)                                         \
WITH labels (n)[1] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[2] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal                                    \
UNION MATCH (n)                                         \
WITH labels (n)[3] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal                                    \
UNION MATCH (n)                                         \
WITH labels (n)[4] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal"  
                                  
graph_test01.run(queryPRB).to_data_frame()

,name,Ntotal
0,MaterialB,78
1,MaterialREF,78
2,Pais,91
3,GrupoAplicacion,88
4,SectorAplicacion,25


In [29]:
#validamos que todos los nodos se han creado (hasta este paso)
queryPRB= "MATCH n=()-[r:INCLUIDO_EN]->()              \
RETURN type(r) as TipoRelacion, count (r) as Ntotal"       
                                  
graph_test01.run(queryPRB).to_data_frame()

,TipoRelacion,Ntotal
0,INCLUIDO_EN,442


# fin proceso - relación  "INCLUIDO_EN" - CREADA

In [30]:
# fin archivo